In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
# for dirname, _, filenames in os.walk('/kaggle/input'):
#     for filename in filenames:
#         print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
from gensim import models,corpora,similarities
import jieba.posseg as pseg
from gensim.models.doc2vec import TaggedDocument,Doc2Vec
import os

In [3]:
def a_sub_b(a,b):
    ret = []
    for el in a:
        if el not in b:
            ret.append(el)
    return ret
stop = [line.strip() for line in open('../input/word2vec-test/stop_words.txt','r',encoding='utf-8').readlines() ]

In [4]:
#读取文件
raw_documents=[]
for root, dirs, files in os.walk(os.path.realpath("/kaggle/input/train-test")):
    for name in files:
        print(os.path.join(root,name))
        try:
            f = open(os.path.join(root, name), 'r',encoding = 'gb18030')
            raw = f.read()
            raw_documents.append(raw)
        except:
            f = open(os.path.join(root, name), 'r',encoding = 'utf-8')
            raw = f.read()
            raw_documents.append(raw)

/kaggle/input/train-test/hongloumeng.txt
/kaggle/input/train-test/xusanguanmaixueji.txt
/kaggle/input/train-test/hetongjiufen.txt
/kaggle/input/train-test/huozhe.txt
/kaggle/input/train-test/zhengfugongzuobaogao.txt
/kaggle/input/train-test/fengkuangxingqisi.txt
/kaggle/input/train-test/xinwen.txt
/kaggle/input/train-test/sharen.txt


In [5]:
#构建语料库
corpora_documents = []
doc=[]            #输出时使用，用来存储未经过TaggedDocument处理的数据，如果输出document，前面会有u
for i, item_text in enumerate(raw_documents):
    print(i)
    words_list=[]
    item=(pseg.cut(item_text))
    for j in list(item):
        words_list.append(j.word)
    words_list=a_sub_b(words_list,list(stop))
    document = TaggedDocument(words=words_list, tags=[i])
    corpora_documents.append(document)
    doc.append(words_list)

Building prefix dict from the default dictionary ...


0


Dumping model to file cache /tmp/jieba.cache
Loading model cost 1.486 seconds.
Prefix dict has been built successfully.


1
2
3
4
5
6
7


In [6]:
#创建model
model = Doc2Vec(vector_size=8, min_count=1, window=5)
model.build_vocab(corpora_documents)
model.train(corpora_documents,total_examples=model.corpus_count,epochs=200)
print(model)

Doc2Vec<dm/m,d8,n5,w5,s0.001,t3>


In [7]:
def str2vec(str):
    test_cut = []
    item = (pseg.cut(str))
    for i in list(item):
        test_cut.append(i.word)
    test_cut = a_sub_b(test_cut,list(stop))
    inferred_vector = model.infer_vector(test_cut)
    return inferred_vector

In [8]:
file_name = []
file_vec = []
for root, dirs, files in os.walk(os.path.realpath("/kaggle/input/doc2vec-test")):
    for name in files:
        print(os.path.join(root,name))
        f = open(os.path.join(root, name), 'r',encoding = 'utf-8')
        raw = f.read()
        temp = str2vec(raw)
        print(temp)
        file_vec.append(temp)
        file_name.append(name)

/kaggle/input/doc2vec-test/xusanguanA.txt
[-8.129585    2.436083    1.958605   -0.13727733 -1.7388843  -4.4480824
 -5.76527    -5.972766  ]
/kaggle/input/doc2vec-test/1989.txt
[ -2.2099679    3.6988096   -7.0868726    2.6197982   -0.35535544
 -12.624043   -10.833429    -4.4218245 ]
/kaggle/input/doc2vec-test/huozheA.txt
[-7.299682    0.25173455 -3.6821916  -1.1604556  -5.1850505  -6.542965
 -0.10048912 -4.138544  ]
/kaggle/input/doc2vec-test/sharenB.txt
[-11.58619    20.889254  -15.96317    12.761768    7.0687     -3.1428607
 -10.539443  -29.95259  ]
/kaggle/input/doc2vec-test/hetongB.txt
[-4.0337334  5.9720306 -7.8722386 -3.1470926 -3.0887308  1.2018747
 -5.9897866 -3.2604723]
/kaggle/input/doc2vec-test/xusanguanB.txt
[-5.6485553   1.3032397  -0.95272833 -0.5810039  -1.735852   -5.5528383
 -1.2615541  -4.679769  ]
/kaggle/input/doc2vec-test/1988.txt
[ -1.5443678    3.1288552   -6.311951     2.001755    -0.47610313
 -11.38113     -8.971995    -3.906674  ]
/kaggle/input/doc2vec-test/huo

In [9]:
temp_vec = []
for i in range(len(file_name)):
    file_name[i] = file_name[i][:-4] + "_vec.txt"
    temp_vec.append([float('{:.6f}'.format(j)) for j in file_vec[i].tolist()])
    with open(file_name[i],'w',encoding="utf-8") as f:
        f.write(str(temp_vec[i]))

In [10]:
file_len = len(file_vec)
print(file_len)
for i in range (file_len):
    for j in range (file_len):
        if(i!=j):
            print(file_name[i],file_name[j],np.linalg.norm(file_vec[i] - file_vec[j]))

12
xusanguanA_vec.txt 1989_vec.txt 14.930192
xusanguanA_vec.txt huozheA_vec.txt 9.489168
xusanguanA_vec.txt sharenB_vec.txt 38.950542
xusanguanA_vec.txt hetongB_vec.txt 13.271565
xusanguanA_vec.txt xusanguanB_vec.txt 6.2679653
xusanguanA_vec.txt 1988_vec.txt 13.454968
xusanguanA_vec.txt huozheB_vec.txt 8.588607
xusanguanA_vec.txt sharenA_vec.txt 41.58625
xusanguanA_vec.txt hongloumengB_vec.txt 13.673809
xusanguanA_vec.txt hongloumengA_vec.txt 12.829943
xusanguanA_vec.txt hetongA_vec.txt 17.275625
1989_vec.txt xusanguanA_vec.txt 14.930192
1989_vec.txt huozheA_vec.txt 15.467754
1989_vec.txt sharenB_vec.txt 36.905136
1989_vec.txt hetongB_vec.txt 16.30351
1989_vec.txt xusanguanB_vec.txt 14.457889
1989_vec.txt 1988_vec.txt 2.6533153
1989_vec.txt huozheB_vec.txt 16.367687
1989_vec.txt sharenA_vec.txt 39.419598
1989_vec.txt hongloumengB_vec.txt 18.566864
1989_vec.txt hongloumengA_vec.txt 17.716509
1989_vec.txt hetongA_vec.txt 18.354467
huozheA_vec.txt xusanguanA_vec.txt 9.489168
huozheA_vec.t

In [11]:
import os
import zipfile
import datetime

def file2zip(packagePath, zipPath):
    '''
  :param packagePath: 文件夹路径
  :param zipPath: 压缩包路径
  :return:
  '''
    zip = zipfile.ZipFile(zipPath, 'w', zipfile.ZIP_DEFLATED)
    for path, dirNames, fileNames in os.walk(packagePath):
        fpath = path.replace(packagePath, '')
        for name in fileNames:
            fullName = os.path.join(path, name)
            name = fpath + '\\' + name
            zip.write(fullName, name)
    zip.close()

In [12]:
packagePath = '/kaggle/working/'
zipPath = '/kaggle/working/output.zip'
if os.path.exists(zipPath):
    os.remove(zipPath)
file2zip(packagePath, zipPath)
print("打包完成")
print(datetime.datetime.utcnow())


打包完成
2023-05-17 11:16:02.214949


In [13]:
# import shutil
# import os
# path = '/kaggle/working/'
# if os.path.exists(path):
#     shutil.rmtree(path)
#     print('删除完成')
# else:
#     print('原本为空')